### Youtube Sentence With Word And Order With Timestamp

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")